In [366]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
import networkx as nx
import matplotlib.colors as mcolors
# warnings.simplefilter(action='ignore', category=FutureWarning)
# pd.options.display.max_rows = None
# pd.options.display.max_columns = None
import glob as glob
import math

#### Both ADNI and A4 used the free surfer recon all command that uses the desikan killany atlas (cerebral cortex into 34 regions)
##### To do: 
- make a list of predefined regions I want to look at and then just harmonize the regional names across the 2 data frames 
- looks like desikan killany atlas is 34 cortical ones but also need to check ones like brainstem etc 

#### Further cleaning: goal is to end up with a4 and adni amy_tau merged csvs with similar naming conventions for each region/col
##### The a4 data has columns with Volume_ prefix so need to drop those 
##### need to makesure that the atlas used between the 2 datasets was consistsent 
- there's also a mean unknown col need to see if that's also present in the adni one 
- also with left right from stanford coumentation "We additionally created bilateral regions taking a volume weighted average across hemispheres. These are provided for in the bi_* columns" so maybe just use these
- need to drop stuff like Mean_non_WM_hypointensities


* additional note- see if the atlas used was consistent between A4 and adni because the naming conventions are different 
* ex mean_ctx_rh_posteriorcingulate in the TAUSUVR vs CTX_RH_PRECENTRAL_SUVR in the UCBERKLEYAV1451_PVC_8mm csv 

In [367]:
#load and analyze a4 data 
merged_a4_at_amypos = pd.read_csv('./a4/merged_a4_at_amy_pos.csv')

In [368]:
print(merged_a4_at_amypos)

           BID         update_stamp_x       ligand  centiloid  \
0    B10423472  2018-12-06 00:00:00.0  Florbetapir    80.8687   
1    B10693248  2018-12-06 00:00:00.0  Florbetapir   113.8210   
2    B10709603  2018-12-06 00:00:00.0  Florbetapir    40.5933   
3    B10904689  2018-12-06 00:00:00.0  Florbetapir   126.6360   
4    B11089276  2018-12-06 00:00:00.0  Florbetapir    35.1012   
..         ...                    ...          ...        ...   
368  B92372599  2019-04-19 00:00:00.0  Florbetapir    40.5933   
369  B97841288  2019-04-19 00:00:00.0  Florbetapir    71.7152   
370  B98032541  2019-04-19 00:00:00.0  Florbetapir    55.2389   
371  B99033399  2019-04-19 00:00:00.0  Florbetapir    53.4082   
372  B99860801  2019-04-19 00:00:00.0  Florbetapir    35.1012   

     Mean_3rd_Ventricle  Mean_4th_Ventricle  Mean_Brain_Stem  \
0              0.845577            0.864285         0.911925   
1              1.110908            0.963089         1.000662   
2              1.041388    

#### notes from a4 doumentation (stanford pipeline)
- created using freesurfer recon- all 
- seems like a4 ones all start with Mean_

In [369]:
#check how many cols have bi_ prefix 
bi_reg_count = sum(col.startswith('bi_') for col in merged_a4_at_amypos.columns)
print(bi_reg_count)
#50 bilateral regions 

50


In [370]:
import pandas as pd

filtered_columns = [col for col in merged_a4_at_amypos.columns if col.startswith('bi_')]
column_list = list(filtered_columns)

# Create a DataFrame from the column_list
a4_col_list = pd.DataFrame({'Column Names': column_list})

# Save the DataFrame to a CSV file
a4_col_list.to_csv("./a4/a4_bi_columns.csv", index=False)

In [371]:
#load and analyze adni data
merged_adni_at_amypos = pd.read_csv("./adni/merged_adni_at_amy_pos.csv")

In [372]:
print(merged_adni_at_amypos)

      RID    SCANDATE TRACER  CENTILOIDS VISCODE VISCODE2    EXAMDATE  \
0      31  2018-04-18    FBP        97.0      y1     m162  2019-04-23   
1     112  2017-07-07    FBP        89.0      y1     m150  2018-08-13   
2     120  2017-09-26    FBP        32.0    init     m132  2017-09-13   
3     142  2015-02-04    FBP       107.0     v41     m120  2016-04-14   
4     210  2018-03-07    FBP        54.0    init     m138  2018-03-13   
..    ...         ...    ...         ...     ...      ...         ...   
406  6572  2022-10-26    FBB       115.0      y2      m24  2020-09-30   
407  6575  2022-11-30    FBB        89.0      y2      m24  2021-01-20   
408  6580  2022-10-13    FBB       142.0      y2      m24  2020-10-09   
409  6581  2022-12-07    FBB        28.0      bl       bl  2018-09-21   
410  6586  2022-10-25    FBB        40.0      y2      m24  2020-10-15   

     META_TEMPORAL_SUVR  META_TEMPORAL_VOLUME  CTX_ENTORHINAL_SUVR  ...  \
0                 1.508               53519.0   

In [373]:
#seems like the volume columns are named like ******_VOLUME so drop these 
#something like endswith "_VOLUME".drop()
drop_cols = [col for col in merged_adni_at_amypos.columns if col.endswith('_VOLUME')]
merged_adni_at_amypos = merged_adni_at_amypos.drop(columns=drop_cols)

In [374]:
#also drop choroid 
#drop adni startswith("CHOROID")
adni_drop_cols = [col for col in merged_adni_at_amypos.columns if col.startswith('CHOROID')]
merged_adni_at_amypos = merged_adni_at_amypos.drop(columns=adni_drop_cols)

In [375]:
#this is without the volume 
print(merged_adni_at_amypos)

      RID    SCANDATE TRACER  CENTILOIDS VISCODE VISCODE2    EXAMDATE  \
0      31  2018-04-18    FBP        97.0      y1     m162  2019-04-23   
1     112  2017-07-07    FBP        89.0      y1     m150  2018-08-13   
2     120  2017-09-26    FBP        32.0    init     m132  2017-09-13   
3     142  2015-02-04    FBP       107.0     v41     m120  2016-04-14   
4     210  2018-03-07    FBP        54.0    init     m138  2018-03-13   
..    ...         ...    ...         ...     ...      ...         ...   
406  6572  2022-10-26    FBB       115.0      y2      m24  2020-09-30   
407  6575  2022-11-30    FBB        89.0      y2      m24  2021-01-20   
408  6580  2022-10-13    FBB       142.0      y2      m24  2020-10-09   
409  6581  2022-12-07    FBB        28.0      bl       bl  2018-09-21   
410  6586  2022-10-25    FBB        40.0      y2      m24  2020-10-15   

     META_TEMPORAL_SUVR  CTX_ENTORHINAL_SUVR  INFERIOR_CEREBGM_SUVR  ...  \
0                 1.508                2.388   

#### further cleaning adni data (only bilateral (l/r hemi regions combined))
- has col names _VOLUME for the volume so drop these 
- they also have _SUVR so maybe could only keep these as a quality control check 
- the naming conventions are stuff like CTX_LH_SUPERIORFRONTAL_SUVR (there's a CTX_LH_SUPERIORFRONTAL_SUVR and CTX_RH_SUPERIORFRONTAL_SUVR for each) 
- looks like there's a CTX_SUPERIORFRONTAL_SUVR and then a CTX_LH_SUPERIORFRONTAL_SUVR and a CTX_RH_SUPERIORFRONTAL_SUVR
- so need to only keep the CTX_SUPERIORFRONTAL_SUVR ones (something like search start with CTX**) if only looking at bilateral brain regions (this correponds to the A4 naming convention "bi_* columns" )


### drop volume before doing anything else

#### Things to drop from A4
- everything starting with Volume 
- ending with 'Ventricle" 
- specific ones ending in _plexus, vessel, _Chiasm, Unknown, _hypointensities, WM, 
- after doing this its 151 rows 

In [376]:
suffixes_to_drop = ['_Ventricle', '_plexus','_vessel','_Chiasm','_hypointensities','WM','CSF','Unknown','_y','_x',"Matter","Vent"]

# Drop columns ending with the specified suffixes
for suffix in suffixes_to_drop:
    a4_drop_cols = [col for col in merged_a4_at_amypos.columns if col.endswith(suffix)]
    merged_a4_at_amypos = merged_a4_at_amypos.drop(columns=a4_drop_cols)

In [377]:
#seems like the volume columns are named like Volume_**** so drop these columns 
#something like starts with "Volume_***".drop()
a4_drop_cols = [col for col in merged_a4_at_amypos.columns if col.startswith('Volume_')]
merged_a4_at_amypos = merged_a4_at_amypos.drop(columns=a4_drop_cols)



In [382]:
merged_a4_at_amypos

,BID,ligand,centiloid,Mean_Brain_Stem,Mean_CC_Anterior,Mean_CC_Central,Mean_CC_Mid_Anterior,Mean_CC_Mid_Posterior,Mean_CC_Posterior,Mean_Left_Accumbens_area,...,bi_precentral,bi_precuneus,bi_rostralanteriorcingulate,bi_rostralmiddlefrontal,bi_superiorfrontal,bi_superiorparietal,bi_superiortemporal,bi_supramarginal,bi_temporalpole,bi_transversetemporal
0,B10423472,Florbetapir,80.8687,0.911925,0.857168,0.733042,0.748900,0.663917,0.866686,1.199217,...,0.955041,1.079061,1.170967,1.084401,1.046561,0.904519,1.056903,0.986939,1.098317,0.949163
1,B10693248,Florbetapir,113.8210,1.000662,0.908223,0.862122,0.873952,0.837348,0.964523,1.289822,...,0.958097,1.114343,1.062407,1.174429,0.979405,0.958415,1.093653,1.146316,1.118879,1.065898
2,B10709603,Florbetapir,40.5933,1.007361,0.853996,0.872356,0.826985,0.782240,0.966260,1.433114,...,0.920471,1.039841,1.029535,0.987615,0.897984,0.900524,1.055535,1.035770,1.069306,0.981632
3,B10904689,Florbetapir,126.6360,0.920387,0.971428,0.823842,0.924871,0.792009,0.918345,1.227138,...,0.912223,1.217753,1.065679,0.993498,1.011460,1.141170,1.077074,1.141837,1.087700,0.963342
4,B11089276,Florbetapir,35.1012,1.035534,0.875824,0.751278,0.736648,0.804294,0.880089,1.459700,...,1.049212,1.160123,1.158528,1.085493,1.048737,1.015478,1.092743,1.098332,1.040324,1.066399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,B92372599,Florbetapir,40.5933,0.919593,0.696205,0.754464,0.687262,0.620278,0.825615,1.338510,...,1.008765,1.071886,0.928695,1.095746,0.997771,1.130347,1.037639,1.122645,1.031807,0.859354
369,B97841288,Florbetapir,71.7152,0.837669,0.714328,0.892816,0.857996,0.848363,1.179848,1.138793,...,1.006039,1.321353,1.060927,1.168246,1.151136,1.190718,1.055261,1.113416,1.074650,0.891608
370,B98032541,Florbetapir,55.2389,1.046703,0.694822,0.912663,0.761793,0.785953,0.889976,1.027593,...,1.061295,1.384344,1.041259,1.635588,1.237379,1.178553,1.106055,1.319375,1.063425,0.829076
371,B99033399,Florbetapir,53.4082,0.940477,0.921984,0.862013,0.859376,0.818269,0.932453,1.190066,...,0.958616,1.109726,1.063435,0.921026,0.994262,0.962612,1.016614,1.041210,1.010308,1.067383


In [378]:
#save the a4 one with just tau suvr and no vent 
merged_a4_at_amypos.to_csv('./a4/merged_a4_at_amypos_novol.csv')
# save the adni one with just tau suvr and no vent 
merged_adni_at_amypos.to_csv('./adni/merged_adni_at_amy_pos_novol.csv')

#### now drop the left and right duplicates 
- also with left right from a4 stanford doumentation "We additionally created bilateral regions taking a volume weighted average across hemispheres. These are provided for in the bi_* columns" so maybe just use these
- in adni data it's going to be "RIGHT_**" OR "LEFT_*" OR "CTX_RH" OR "CTX_LH" 
- for adni some regions like orbiotfrontal gyrus, there are 3: orbiotfrontal gyrus, LEFT_ORBITOFRONTAL_GYRUS, and RIGHT_ etc so just take the one without left and right 


In [379]:
#find the adni brain regions list without "RIGHT_**" OR "LEFT_*" OR "CTX_RH" OR "CTX_LH" 
prefixes_to_remove = ['RIGHT_', 'LEFT_', 'CTX_RH', 'CTX_LH']
for prefix in prefixes_to_remove:
    adni_drop_cols = [col for col in merged_adni_at_amypos.columns if col.startswith(prefix)]
    merged_adni_at_amypos = merged_adni_at_amypos.drop(columns=adni_drop_cols)
#this leaves 47 cols 

In [380]:
print(merged_adni_at_amypos)

      RID    SCANDATE TRACER  CENTILOIDS VISCODE VISCODE2    EXAMDATE  \
0      31  2018-04-18    FBP        97.0      y1     m162  2019-04-23   
1     112  2017-07-07    FBP        89.0      y1     m150  2018-08-13   
2     120  2017-09-26    FBP        32.0    init     m132  2017-09-13   
3     142  2015-02-04    FBP       107.0     v41     m120  2016-04-14   
4     210  2018-03-07    FBP        54.0    init     m138  2018-03-13   
..    ...         ...    ...         ...     ...      ...         ...   
406  6572  2022-10-26    FBB       115.0      y2      m24  2020-09-30   
407  6575  2022-11-30    FBB        89.0      y2      m24  2021-01-20   
408  6580  2022-10-13    FBB       142.0      y2      m24  2020-10-09   
409  6581  2022-12-07    FBB        28.0      bl       bl  2018-09-21   
410  6586  2022-10-25    FBB        40.0      y2      m24  2020-10-15   

     META_TEMPORAL_SUVR  CTX_ENTORHINAL_SUVR  INFERIOR_CEREBGM_SUVR  ...  \
0                 1.508                2.388   

In [381]:
#now drop everything accet the but the a4 bi_ cols 
a4_bi_cols = col.startswith('bi_')

NameError: name 'col' is not defined

In [ ]:
#check how many l/r hemi combined or bilateral regions there are 
#check how many cols have bi_ prefix 
adni_bi_reg_count = sum(col.startswith('CTX_') for col in merged_adni_at_amypos.columns)
print(adni_bi_reg_count)
#how are there 156 bilateral/combined LR hemi regions for adni and then only 50 for a4 
#because the LR ones are CTX_LR etc so need to eclude 

26


In [ ]:
#count ctx_ cols 52 ctx cols 
import re

adni_bi_reg_count = sum(col.startswith('CTX_') and not re.search(r'(LH_|RH_)', col) for col in merged_adni_at_amypos.columns)
print(adni_bi_reg_count)

26


In [ ]:
#count ctx cols and then save as csv to read them 
import re
import pandas as pd

#cols that start with CTX_ and don't have LH or RH in them 
filtered_columns = [col for col in merged_adni_at_amypos.columns if col.startswith('CTX_') and not re.search(r'(LH_|RH_)', col)]

# Create a DataFrame from the filtered column names
adni_col_list = pd.DataFrame({'Column Names': filtered_columns})

# Save the DataFrame to a CSV file
adni_col_list.to_csv("./adni/adni_bi_colummns.csv", index=False)


In [ ]:
print(adni_col_list)

                         Column Names
0                 CTX_ENTORHINAL_SUVR
1                   CTX_BANKSSTS_SUVR
2    CTX_CAUDALANTERIORCINGULATE_SUVR
3                     CTX_CUNEUS_SUVR
4                   CTX_FUSIFORM_SUVR
5           CTX_INFERIORPARIETAL_SUVR
6           CTX_INFERIORTEMPORAL_SUVR
7                     CTX_INSULA_SUVR
8           CTX_ISTHMUSCINGULATE_SUVR
9           CTX_LATERALOCCIPITAL_SUVR
10                   CTX_LINGUAL_SUVR
11            CTX_MIDDLETEMPORAL_SUVR
12               CTX_PARACENTRAL_SUVR
13           CTX_PARAHIPPOCAMPAL_SUVR
14             CTX_PERICALCARINE_SUVR
15               CTX_POSTCENTRAL_SUVR
16        CTX_POSTERIORCINGULATE_SUVR
17                CTX_PRECENTRAL_SUVR
18                 CTX_PRECUNEUS_SUVR
19  CTX_ROSTRALANTERIORCINGULATE_SUVR
20           CTX_SUPERIORFRONTAL_SUVR
21          CTX_SUPERIORPARIETAL_SUVR
22          CTX_SUPERIORTEMPORAL_SUVR
23             CTX_SUPRAMARGINAL_SUVR
24              CTX_TEMPORALPOLE_SUVR
25        CT

In [ ]:
print(a4_col_list)

                   Column Names
0             bi_Accumbens_area
1                   bi_Amygdala
2                    bi_Caudate
3          bi_Cerebellum_Cortex
4    bi_Cerebellum_White_Matter
5      bi_Cerebral_White_Matter
6                bi_Hippocampus
7               bi_Inf_Lat_Vent
8          bi_Lateral_Ventricle
9                   bi_Pallidum
10                   bi_Putamen
11                  bi_Thalamus
12                 bi_VentralDC
13            bi_choroid_plexus
14                    bi_vessel
15                  bi_bankssts
16   bi_caudalanteriorcingulate
17       bi_caudalmiddlefrontal
18                    bi_cuneus
19                bi_entorhinal
20               bi_frontalpole
21                  bi_fusiform
22          bi_inferiorparietal
23          bi_inferiortemporal
24                    bi_insula
25          bi_isthmuscingulate
26          bi_lateraloccipital
27      bi_lateralorbitofrontal
28                   bi_lingual
29       bi_medialorbitofrontal
30      